In [1]:
import pandas as pd
import geopandas
import json
import os
from shapely.geometry import Point
import difflib
from geopy.distance import vincenty
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
inline_rc = dict(mpl.rcParams)

In [2]:
in_folder = "audits/2019_02/data/"
out_folder = "audits/2019_02/images/"
in_folder = "../data/"
out_folder = "/tmp"

In [3]:
# on fusionne les lignes et les parcours, en conservant les ref STIF

In [4]:
osm_lines = pd.read_csv(os.path.join(in_folder, "osm-transit-extractor_lines.csv"), dtype=str)
osm_lines = osm_lines[osm_lines['mode'] == 'bus']


In [5]:
osm_line_route = pd.read_csv(os.path.join(in_folder, "osm-transit-extractor_line_routes.csv"), dtype=str)
osm_lines.rename(columns={'colour': 'line_colour'}, inplace=True)

result = pd.merge(osm_lines,osm_line_route, on='line_id')

osm_lines = result[['route_id', 'osm:ref:FR:STIF:ExternalCode_Line', 'line_id', 'name', 'code', 'line_colour', 'operator', 'network']].copy()

In [6]:
osm_routes = pd.read_csv(os.path.join(in_folder, "osm-transit-extractor_routes.csv"), dtype=str)
osm_routes = osm_routes[['route_id', 'colour']].copy()
osm_routes.rename(columns={'colour': 'route_colour'}, inplace=True)

result = pd.merge(osm_lines,osm_routes, on='route_id')
osm_lines = result[['osm:ref:FR:STIF:ExternalCode_Line', 'line_id', 'name', 'code', 'line_colour', 'route_colour', 'operator', 'network']].copy()

In [7]:
def compute_colour(row):
    if pd.isna(row['line_colour']):
        colour = row['route_colour']
    else :     
        colour = row['line_colour']
    return colour

osm_lines['colour'] = osm_lines.apply(compute_colour, axis=1)

In [8]:
osm_lines.dropna(inplace=True)
osm_lines = osm_lines[['osm:ref:FR:STIF:ExternalCode_Line', 'line_id', 'name', 'code', 'colour', 'operator', 'network']].copy()
osm_lines.drop_duplicates(inplace=True) #jusque là, on avait des parcours, là, on retombe bien sur des lignes
len(osm_lines)

486

In [9]:
# GTFS

In [10]:
gtfs_lines = pd.read_csv(os.path.join(in_folder, "gtfs_routes.txt"), dtype=str)
gtfs_lines = gtfs_lines[gtfs_lines['route_type'] == '3']
gtfs_lines['colour'] = "#" + gtfs_lines['route_color']
gtfs_lines = gtfs_lines[['route_id','colour']].copy()
gtfs_lines = gtfs_lines[gtfs_lines['colour']!='#000000'].copy()
len(gtfs_lines)

1710

In [11]:
# on fusionne

In [12]:
gtfs_lines.rename(columns={'colour': 'gtfs_colour'}, inplace=True)
osm_lines.rename(columns={'colour': 'osm_colour'}, inplace=True)


result = pd.merge(osm_lines,gtfs_lines, right_on='route_id', left_on='osm:ref:FR:STIF:ExternalCode_Line')
len(result)

466

In [13]:
#result[result['osm_colour']!=result['gtfs_colour']]
# TODO : faire un calcul de similarité de couleur ?